<a href="https://colab.research.google.com/github/jbarrasa/goingmeta/blob/main/session23/python_notebooks/GM23_ArtGalleryAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j openai langchain tiktoken

In [16]:
import os
from pprint import pprint
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['NEO4J_URL'] = ""
os.environ['NEO4J_USR'] = ""
os.environ['NEO4J_PWD'] = ""

In [4]:
from langchain.graphs import Neo4jGraph
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

url = os.environ['NEO4J_URL']
username = os.environ['NEO4J_USR']
password = os.environ['NEO4J_PWD']

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='artworks',
    node_label="Artwork",
    text_node_properties=['title', 'display_caption', 'artist'],
    embedding_node_property='embedding',
)


In [8]:
search_string = "I'm interested in works by Constable featuring buildings"
#search_string = "How does Andy Warhol's artwork address the theme of death?"
#search_string = "I'm interested in artworks that focus on geometric forms"

response = vector_index.similarity_search(search_string, k=3)
for r in response:
  print(r.page_content)



title: A Bank on Hampstead Heath
display_caption: 
Between 1819 to 1826 Constable rented a house at Hampstead nearly every summer and made many oil sketches of the surrounding landscape and skies. In 1827 he moved there permanently. Constable was fascinated with the broken textures of the heath, and made a number of studies of it seen from close to. Some of these are quite highly finished, like this example. A glimpse of more distant landscape to the right acts as a counterbalance, while the tiny figures on the horizon help establish the scale. 
Gallery label, May 2007
Does this text contain inaccurate information or language that you feel we should improve or change? We would like to hear from you.

artist: Constable, John

title: The Church Porch, East Bergholt
display_caption: 

Three figures sit and talk outside the church in East Bergholt, the village in Suffolk where Constable was born. A man, wearing a red cloak and tricorn hat, and a woman lean on the tombs while a younger gir

In [5]:
enrich_for_summary_query = """
match (node:Artwork)-[:has_subject|broader*]->(s)
with node as aw, reduce(r="", x in collect(s.name) | r + ", " + x) as featurelist, score, {} as metadata
return "\n\ntitle: " + aw.title + "\nauthor: " + aw.artist + "\nmedium: " + aw.medium + "\nfeatures: " + featurelist + "\nurl:" + aw.url as text, score, metadata
"""

contextualized_vectorstore = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(),
    url=os.environ['NEO4J_URL'],
    username=os.environ['NEO4J_USR'],
    password=os.environ['NEO4J_PWD'],
    index_name="artworks",
    retrieval_query=enrich_for_summary_query,
)

In [6]:
def kg_recommendations_app(input):
  response = contextualized_vectorstore.similarity_search(input, k=3)
  return "\n\n".join([d.page_content for d in response])

In [9]:
print(kg_recommendations_app(search_string))



title: A Bank on Hampstead Heath
author: Constable, John
medium: Oil paint on canvas
features: , townscapes / man-made features, path, architecture, UK London, places, figure, UK countries and regions, England, adults, people, Hampstead Heath - non-specific, groups, group
url:http://www.tate.org.uk/art/artworks/constable-a-bank-on-hampstead-heath-n02658



title: The Church Porch, East Bergholt
author: Constable, John
medium: Oil paint on canvas
features: , people, UK countries and regions, UK cities, towns and villages, England, architecture, East Anglia, emotions, concepts and ideas, figure, UK counties, emotions and human qualities, literature and fiction, adults, Gray, Thomas, ‘Elegy Written in a Country Churchyard’, 1751, religious, church, path, Suffolk, townscapes / man-made features, contemplation, East Bergholt, St Mary’s Church, graveyard, England, East, places, literature (not Shakespeare)
url:http://www.tate.org.uk/art/artworks/constable-the-church-porch-east-bergholt-n01

## Prompt Definition

In [10]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

general_system_template_app = '''
You are a personal assistant for an art gallery.
write an email to {customerName}, one of your members, to plan their next visity to the gallery.
The email should summarize the artworks that match what they searched emphasizing on features and medium.
Introduce an art pun too based on the results!
Please only choose from the Artworks listed below. Do not come up with or add any new elements to the list.
Each artwork description comes with a "url" field.
Make sure to link to the url with descriptive name text for each artwork so the customer can easily find them.

---
# Relevant Artworks:
{recArtworks}
---
'''

general_user_template_app = '''{searchPrompt}'''
messages_app = [
    SystemMessagePromptTemplate.from_template(general_system_template_app),
    HumanMessagePromptTemplate.from_template(general_user_template_app),
]
prompt_app = ChatPromptTemplate.from_messages(messages_app)

In [14]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain.schema import StrOutputParser

llm = ChatOpenAI()

chain_app2 = ({'recArtworks': itemgetter('searchPrompt') | RunnableLambda(kg_recommendations_app),
             'customerName': lambda x:x['customerName'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt_app
            | llm
            | StrOutputParser())

In [15]:
customer_name = "Alex Erdl"

pprint(chain_app2.invoke({'searchPrompt': search_string, 'customerName': customer_name }))

("Subject: Plan your visit to the gallery - Constable's artworks featuring "
 'buildings!\n'
 '\n'
 'Dear Alex Erdl,\n'
 '\n'
 'I hope this email finds you well. I wanted to reach out and let you know '
 'that we have some amazing artworks by Constable that align with your '
 "interest in works featuring buildings. I've selected a few pieces that I "
 "think you'll enjoy during your next visit to the gallery.\n"
 '\n'
 '1. "A Bank on Hampstead Heath" - This captivating piece showcases a '
 'townscape with man-made features, architecture, and a path. The painting '
 'beautifully captures the essence of UK London, with its vibrant colors and '
 'attention to detail. You can find this artwork '
 '[here](http://www.tate.org.uk/art/artworks/constable-a-bank-on-hampstead-heath-n02658).\n'
 '\n'
 '2. "The Church Porch, East Bergholt" - In this artwork, Constable skillfully '
 'portrays a church and its surroundings. The painting features figures and '
 "the architecture of St Mary's Church in